# MG LSTM

In [1]:
import glob
from music21 import converter, instrument, note, chord, stream
import numpy as np
import pickle
import datetime
import re
import sys

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from keras import backend as K

print(sys.executable)
print(K.tensorflow_backend._get_available_gpus())

/home/cipher000/anaconda3/envs/tensorflow1.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/cipher000/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


/home/cipher000/anaconda3/envs/tensorflow1.4/bin/python
[]


In [2]:
output_notes_filepath = '../data/output_notes'
midi_files = '../data/MidiWorld/Pop/AceofBase-ThatSheWants.mid'
# weights_file = '../weights/lstm_weights.hdf5'



output_name = midi_files.split('/')[-2]

timestamp = str(datetime.datetime.now()).split()[0].replace('-','')

In [3]:
sequence_length = 100 # the lstm will predict the next note based on the last set of notes heard
node1 = 512
node2 = 256
drop = 0.3
epochs = 1 # 200
batch_size = 64
notes_generated = 500

### Training Functions

In [4]:
def convert_to_notes():
    notes = [] # list of notes and chords
    cnt = 0
    
    print("\n**Loading Midi files**")
    for file in glob.glob(midi_files): # loading midi filepaths
        print(file)
        try:
            midi = converter.parse(file) # midi type music21.stream.Score
            parts = instrument.partitionByInstrument(midi) # parts type music21.stream.Score

            if parts: 
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes 
            # notes_to_parse type music21.stream.iterator.RecursiveIterator
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    to_append = '.'.join(str(n) for n in element.normalOrder)
                    notes.append(to_append)
            cnt +=1
        except Exception as e:
            print(e)
            pass
#     with open(notes_file, 'wb') as filepath:
#         pickle.dump(notes, filepath)
    n_vocab = len(set(notes))
    print("Notes Converted.\n# Midi Files: {} # Notes: {} # Unique Notes: {}".format(cnt,len(notes),n_vocab))
    return notes, n_vocab

def prep_input(notes, n_vocab):
    print("\n**Preparing sequences for training**")
    pitchnames = sorted(set(item for item in notes)) # list of unique chords and notes
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) # enumerate pitchnames in dict
    
    network_input = []
    network_output = []
    
    # i equals total notes less declared sequence length of LSTM (ie 5000 - 100)
    # sequence input for each i is list of notes i to end of sequence length (ie 0-100 for i = 0)
    # sequence output for each i is single note at i + sequence length (ie 100 for i = 0)
    for i in range(0,len(notes) - sequence_length, 1): 
        sequence_in = notes[i:i + sequence_length] # 100
        sequence_out = notes[i + sequence_length] # 1
        
        # enumerate notes and chord sequences with note_to_int enumerated encoding
        # network input/output is a list of encoded notes and chords based on note_to_int encoding
        # if 100 unique notes/chords, the encoding will be between 0-100
        input_add = [note_to_int[char] for char in sequence_in]
        network_input.append(input_add) # sequence length
        output_add = note_to_int[sequence_out]
        network_output.append(output_add) # single note
        
    n_patterns = len(network_input) # notes less sequence length
    print("Pitchnames: {}".format(pitchnames))
    print("Inputting {} encoded notes/chords into network; sequence length {}".format(n_patterns,sequence_length))

    print("\n**Reshaping for training**")
    net_input_len = len(network_input)
    # convert network input/output from lists to numpy arrays
    # reshape input to (notes less sequence length, sequence length)
    # reshape output to (notes less sequence length, unique notes/chords)
    network_input = np.reshape(network_input, (net_input_len, sequence_length, 1))
    network_output = np_utils.to_categorical(network_output)
    print("Shapes. Network Input: {} Network Output: {}".format(network_input.shape, network_output.shape))
    return pitchnames, network_input, network_output

def create_network(network_input, n_vocab,weights_file=None):
    print("\n**LSTM model initializing**")
    model = Sequential()
    # TODO determine layers - 2 or 3?
    # Layer 1
    model.add(LSTM(node1,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(drop))
    # Layer 2
    model.add(LSTM(node1, return_sequences=True))
    model.add(Dropout(drop))
    # Layer 3
    model.add(LSTM(node1))
    model.add(Dense(node2))
    model.add(Dropout(drop))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    if weights_file:
        model.load_weights(weights_file)
        print("LSTM model initialized for midi creation with weights from {}".format(weights_file))
    else:
        print("LSTM model initialized for training (no weights file)")
    
    return model

def train(model, network_input, network_output):
    print("\n**Training LSTM network**")
    filepath = "../weights/weights-{epoch:02d}-{loss:.4f}.hdf5"
    
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min')
    
    callbacks_list = [checkpoint]
    
    print("Fitting Model. \nNetwork Input Shape: {} Network Output Shape: {}".format(network_input.shape,network_output.shape))
    print("Epochs: {} Batch Size: {}".format(epochs, batch_size))
    
    model.fit(
        network_input, 
        network_output, 
        epochs=epochs, 
        batch_size=batch_size, 
        callbacks=callbacks_list)
    weights_file = '../weights/{}-{}-lstm_weights.hdf5'.format(timestamp, output_name)
    model.save_weights(weights_file)
    print("LSTM training complete - weights saved at: {}".format(weights_file))
    return weights_file

### Create MIDI Functions

In [5]:
def prep_output_sequences(notes, pitchnames, n_vocab):
#     TODO determine if this function is duplicative
    print("\nPreparing sequences for output")
 
    network_input = []
    network_output = []
    
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    print("Note to Int: {}".format(note_to_int))
    
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_patterns = len(network_input)
    
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    normalized_input = normalized_input / float(len(pitchnames))
    
    return network_input, normalized_input

# def reshape_for_creation(network_input, sequence_length,pitchnames):
#     print("\n**Reshaping for midi creation**")
#     net_input_len = len(network_input)
#     # convert network input/output from lists to numpy arrays
#     # reshape input to (notes less sequence length, sequence length)
#     # reshape output to (notes less sequence length, unique notes/chords)
#     normalized_input = np.reshape(network_input, (net_input_len, sequence_length, 1))
#     normalized_input = normalized_input / float(len(pitchnames))
#     print("Shapes. Network Input: {} Normalized Input: {}".format(network_input.shape, normalized_input.shape))
#     return normalized_input

def generate_notes(model, network_input, pitchnames, n_vocab):
    print("\n**Generating notes**")
    start = np.random.randint(0,len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames)) # convert integers back to notes
    
    pattern = network_input[start] # randomly instantiate with single number from 0 to length of network input
    prediction_output = []
    
    # for each note in notes generated declared as hyperparameter above (ie 500)
    for note_index in range(notes_generated): 
#         print("Note Index: {}".format(note_index))
        prediction_input = np.reshape(pattern, (1,len(pattern),1))
#         print("Prediction Input Shape: {}".format(prediction_input.shape))
        prediction_input = prediction_input / float(n_vocab)
#         print("Prediction Input Shape after dividing by unique chords/notes: {}".format(prediction_input.shape))
        
        prediction = model.predict(prediction_input, verbose=0)
#         print("Prediction: {}".format(prediction))
        
        index = np.argmax(prediction)
#         print("Index: {}".format(index))
        result = int_to_note[index]
#         print("Result: {}".format(result))
        prediction_output.append(result)
        
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
#         print("Pattern: {}".format(pattern))
    print("Prediction Output: {}".format(prediction_output))
        
    return prediction_output

def create_midi(prediction_output,output_name, epochs):
    print("\n**Creating midi**")
    offset = 0
    output_notes = []
    
    for pattern in prediction_output:
        sound = instrument.Flute() # declare a Music21 package instrument
        # prepares chords (if) and notes (else)
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = sound 
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = sound 
            output_notes.append(new_note)
        
        offset += 0.5
        
    midi_stream = stream.Stream(output_notes)
    output_file = '../output/{}-{}-lstm_midi-{}.mid'.format(timestamp,output_name,epochs)
    midi_stream.write('midi',fp=output_file)
    print("Midi saved at: {}".format(output_file))
    with open(output_notes_filepath, 'wb') as f:
        pickle.dump(output_notes, f)
    return output_notes, midi_stream

### LSTM Model Execution

In [10]:
def train_create_midi():
    # Execute all functions from training to midi creation
    # Midi preparation
    notes, n_vocab = convert_to_notes() 
    pitchnames, network_input, network_output = prep_input(notes, n_vocab) 
    # LSTM training
    model = create_network(network_input, n_vocab)
    weights_file = train(model, network_input, network_output)
    # Midi creation
    normalized_input = prep_output_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, pitchnames,weights_file)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    output_notes, midi = create_midi(prediction_output,output_name,epochs)

In [11]:
train_create_midi()


**Loading Midi files**
../data/MidiWorld/Pop/AceofBase-ThatSheWants.mid
Notes Converted.
# Midi Files: 1 # Notes: 992 # Unique Notes: 32

**Preparing sequences for training**
Pitchnames: ['1.4.8', '1.5.8', '1.6', '11.3.6', '6.10.1', '6.9.1', '8.0.3', 'A3', 'A4', 'A5', 'B-3', 'B-4', 'B-5', 'B2', 'B4', 'C#3', 'C#4', 'C#5', 'C#6', 'C2', 'C3', 'C4', 'C5', 'E-5', 'E5', 'F#3', 'F#5', 'F5', 'G#2', 'G#3', 'G#4', 'G#5']
Inputting 892 encoded notes/chords into network; sequence length 100

**Reshaping for training**
Shapes. Network Input: (892, 100, 1) Network Output: (892, 32)

**LSTM model initializing**
LSTM model initialized for training (no weights file)

**Training LSTM network**
Fitting Model. 
Network Input Shape: (892, 100, 1) Network Output Shape: (892, 32)
Epochs: 1 Batch Size: 64
Epoch 1/1
892/892 [==============================] - 42s 47ms/step - loss: 3.6359
LSTM training complete - weights saved at: ../weights/20180323-Pop-lstm_weights.hdf5

Preparing sequences for output
Note to

AttributeError: 'tuple' object has no attribute 'shape'

### Resources

Model adapted from Sigurour Skuli's [How to Generate Music using a LSTM Neural Network in Keras](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)